In [1]:
import hashlib
import requests
import feedparser
from bs4 import BeautifulSoup

In [13]:
def scrape_website(url):
    """Attempts to scrape article content from a website"""
    headers = {'User-Agent': 'Mozilla/5.0'} #mimics web browser request
    try:
        response = requests.get(url, headers=headers, timeout=5)
        if response.status_code != 200:
            return None

        # To-Do: generalize to any website
        soup = BeautifulSoup(response.text, 'html.parser')
        title = soup.title.string if soup.title else "No Title"
        main_content = soup.find("article") or soup.find("div", class_="content")
        paragraphs = main_content.find_all("p") if main_content else soup.find_all("p")
        content = "\n".join([p.get_text() for p in paragraphs])

        data = {
            "title": title,
            "link": url,
            "content": content[:1000],  #limit preview to save disk space?
        }
        return data
    except requests.RequestException as error:
        print(f"Error scraping {url}: {error}")
        return None

In [14]:
site = "https://techcrunch.com"
scraped_article = scrape_website(site)

In [15]:
scraped_article

{'title': 'TechCrunch | Startup and Technology News',
 'link': 'https://techcrunch.com',
 'content': '\n\n\t\tLatest\t\n\n\n\n\t\tAI\t\n\n\n\n\t\tAmazon\t\n\n\n\n\t\tApps\t\n\n\n\n\t\tBiotech & Health\t\n\n\n\n\t\tClimate\t\n\n\n\n\t\tCloud Computing\t\n\n\n\n\t\tCommerce\t\n\n\n\n\t\tCrypto\t\n\n\n\n\t\tEnterprise\t\n\n\n\n\t\tEVs\t\n\n\n\n\t\tFintech\t\n\n\n\n\t\tFundraising\t\n\n\n\n\t\tGadgets\t\n\n\n\n\t\tGaming\t\n\n\n\n\t\tGoogle\t\n\n\n\n\t\tGovernment & Policy\t\n\n\n\n\t\tHardware\t\n\n\n\n\t\tInstagram\t\n\n\n\n\t\tLayoffs\t\n\n\n\n\t\tMedia & Entertainment\t\n\n\n\n\t\tMeta\t\n\n\n\n\t\tMicrosoft\t\n\n\n\n\t\tPrivacy\t\n\n\n\n\t\tRobotics\t\n\n\n\n\t\tSecurity\t\n\n\n\n\t\tSocial\t\n\n\n\n\t\tSpace\t\n\n\n\n\t\tStartups\t\n\n\n\n\t\tTikTok\t\n\n\n\n\t\tTransportation\t\n\n\n\n\t\tVenture\t\n\n\n\n\t\tEvents\t\n\n\n\n\t\tStartup Battlefield\t\n\n\n\n\t\tStrictlyVC\t\n\n\n\n\t\tNewsletters\t\n\n\n\n\t\tPodcasts\t\n\n\n\n\t\tVideos\t\n\n\n\n\t\tPartner Content\t\n\n\n\n\t\tTec